# CLIP Trained from scratch on Memotion and also finetuned on Memotion sentences



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q timm
!pip install -q transformers

     |████████████████████████████████| 376 kB 1.7 MB/s 
     |████████████████████████████████| 2.8 MB 1.9 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 636 kB 76.5 MB/s 
     |████████████████████████████████| 3.3 MB 60.7 MB/s 
     |████████████████████████████████| 895 kB 77.2 MB/s 


In [ ]:
# ! cp './drive/MyDrive/Wipro/Data/memotion1_images_combined.zip' .
# ! unzip -q memotion1_images_combined.zip
# ! rm -rf memotion1_images_combined.zip



! cp './drive/MyDrive/Wipro/Final/Data/Memotion/images.zip' .
! unzip -q images.zip
! rm -rf images.zip

In [ ]:
import shutil
import torch
from torch.utils.data import DataLoader,Dataset
import transformers
from transformers import CLIPModel, CLIPConfig
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
import os 
import pandas as pd
import cv2 as cv
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import WeightedRandomSampler
from sklearn.metrics import  f1_score
from tqdm.autonotebook import tqdm

from torchsummary import summary

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


from tensorboard.plugins import projector

from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import tqdm 

from sklearn.utils.class_weight import compute_class_weight


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# CLIP Tokeniser
tokeniser = transformers.CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

Downloading:   0%|          | 0.00/862k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/568 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


# Labels Preparation

In [ ]:
df = pd.read_csv('./drive/MyDrive/Wipro/Final/Data/Memotion/csv/memotion_train_sentences.csv')

In [ ]:
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,look there my friend lightyear now all sohalik...,3,1,0,0,2
1,image_2.jpeg,the best of #10 yearchallenge! completed in le...,0,1,0,1,2
2,image_3.JPG,sam thorne @strippin ( follow follow saw every...,2,0,0,0,2
3,image_4.png,10 year challenge - sweet dee edition,2,2,2,1,2
4,image_5.png,10 year challenge with no filter 47 hilarious ...,3,3,2,0,1


In [ ]:

# Remove Images that don't exist

rem_images = ['image_1567.jpg','image_4924.jpg','image_5119.png','image_6357.jpg']
df_word_cleaned = df.drop(df[df['image_name'].isin(rem_images)].index)
df_word_cleaned.dropna(inplace=True)
df_word_cleaned.index = range(df_word_cleaned.shape[0])


In [ ]:
df_word_cleaned.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,look there my friend lightyear now all sohalik...,3,1,0,0,2
1,image_2.jpeg,the best of #10 yearchallenge! completed in le...,0,1,0,1,2
2,image_3.JPG,sam thorne @strippin ( follow follow saw every...,2,0,0,0,2
3,image_4.png,10 year challenge - sweet dee edition,2,2,2,1,2
4,image_5.png,10 year challenge with no filter 47 hilarious ...,3,3,2,0,1


In [ ]:
def apply_labels(x):
  
  if x>=1:
    return 1

  else:
    return 0

In [ ]:
df_word_cleaned['humour'] = df_word_cleaned['humour'].apply(apply_labels)
df_word_cleaned['sarcasm'] = df_word_cleaned['sarcasm'].apply(apply_labels)
df_word_cleaned['offensive'] = df_word_cleaned['offensive'].apply(apply_labels)

# MTL Model

In [ ]:
'''
  Dataset class
'''

class MemotionDataTaskB(Dataset):
    

    def __init__(self,root_dir,img_names,ocr_text,class_labels,tokeniser,transforms=None):
        
        self.img_names = img_names
        self.ocr_text = ocr_text
        self.class_labels = class_labels
        self.tokeniser = tokeniser
        self.root_dir = root_dir
        self.transforms = transforms
        
    def __len__(self):
        return len(self.img_names)
        
        
    def __getitem__(self,idx):
        
        # print(self.img_names[idx])

        img = cv.imread(os.path.join(self.root_dir,self.img_names[idx]))
        img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        
        if self.transforms:
            img = self.transforms(img)
        
        output_token_ids = self.tokeniser.encode_plus(
            self.ocr_text[idx],
            max_length=76,
            add_special_tokens=True,
            padding='max_length',
            return_tensors='pt',
            return_attention_mask=True,
            truncation = True
        )
        
        return {
            'img': torch.FloatTensor(img),
            'input_ids': output_token_ids['input_ids'],
            'att_mask': output_token_ids['attention_mask'],
            'emotion_humour': torch.tensor(self.class_labels[idx,0],dtype=torch.long),
            'emotion_sarcasm': torch.tensor(self.class_labels[idx,1],dtype=torch.long),
            'emotion_offensive': torch.tensor(self.class_labels[idx,2],dtype=torch.long),
            'emotion_motivational': torch.tensor(self.class_labels[idx,3],dtype=torch.long)
        }
        


def create_dataset(df,tokeniser,max_len=76,train=True):
    ds = MemotionDataTaskB(
        root_dir = './train_images/' if train else './test_images' ,
        img_names = df['image_name'].to_list(),
        ocr_text= df['text_corrected'].to_list(),
        class_labels = df.loc[:, 'humour': 'motivational'].values,
        tokeniser = tokeniser,
        transforms = torchvision.transforms.Compose(
        [
                torchvision.transforms.ToPILImage(),
                torchvision.transforms.Resize((224,224)),
                torchvision.transforms.ToTensor(),
        ]
        )
    )
    return ds        

In [ ]:
'''
  To create model based on CLIP 
'''
class MemotionModel(nn.Module):
    def __init__(self,scratch=True):
        super(MemotionModel,self).__init__()
        # self.pre_model = CLIPModel(CLIPConfig()).to(device)
        self.pre_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        self.scratch = scratch
        
        if scratch:
          for params in self.pre_model.parameters():
              params.requires_grad = True
        
        else:
          for params in self.pre_model.parameters():
              params.requires_grad = False
                
    def forward(self,x,input_ids,att_mask):
        img_embed =  self.pre_model.get_image_features(x)
        text_embed = self.pre_model.get_text_features(input_ids.squeeze(1),attention_mask=att_mask.squeeze(1))
        return img_embed, text_embed
        

In [ ]:
class MtlModel(nn.Module):
    def __init__(self, basic_model, num_class=[1,1,1,1]):
        super(MtlModel, self).__init__()
        self.basic_model = basic_model

        for param in self.basic_model.parameters():
          param.requires_grad = False

        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,num_class[0])
        self.fc3 = nn.Linear(512,num_class[1])
        self.fc4 = nn.Linear(512,num_class[2])
        self.fc5 = nn.Linear(512,num_class[3])
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
        self.bn = nn.BatchNorm1d(512)

    def forward(self,x,input_ids,att_mask):        
        img_embed, text_embed =  self.basic_model(x,input_ids.squeeze(1),att_mask.squeeze(1))
        x = torch.cat((img_embed,text_embed),dim=1)
        # if not self.scratch:
        # x.requires_grad = True
        x = self.fc1(x)
        x = self.bn(x)
        
        out1  = self.fc2(x)
        out2  = self.fc3(x)
        out3  = self.fc4(x)
        out4  = self.fc5(x)
        # x = self.softmax(x)


        return out1, out2, out3, out4

In [ ]:
'''
  Return Weights for training
'''

def ret_class_weights(df_new,train_dataset):

  # class_weights_h = list(compute_class_weight('balanced', np.unique(df_new['humour'].values), df_new['humour'].values))
  # class_weights_s = list(compute_class_weight('balanced', np.unique(df_new['sarcasm'].values), df_new['sarcasm'].values))
  # class_weights_o = list(compute_class_weight('balanced', np.unique(df_new['offensive'].values), df_new['offensive'].values))
  # class_counts = list(np.unique(df_new['motivational'],return_counts=True)[1])
  # class_weights_m = [class_counts[0]/class_counts[1]]

  # return class_weights_h, class_weights_s, class_weights_o, class_weights_m

  class_counts = list(np.unique(df_new['humour'],return_counts=True)[1])
  class_weights_h = [class_counts[0]/class_counts[1]]

  class_counts = list(np.unique(df_new['sarcasm'],return_counts=True)[1])
  class_weights_s = [class_counts[0]/class_counts[1]]

  class_counts = list(np.unique(df_new['offensive'],return_counts=True)[1])
  class_weights_o = [class_counts[0]/class_counts[1]]

  class_counts = list(np.unique(df_new['motivational'],return_counts=True)[1])
  class_weights_m = [class_counts[0]/class_counts[1]]

  return class_weights_h, class_weights_s, class_weights_o, class_weights_m

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=21)

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/Wipro/Final/Base_Models/models/clip_scratch_memotion_sentences_basic_model.pt'

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 256
cur_split = 1
softmax_layer = nn.Softmax(dim=1)
sigmoid_layer = nn.Sigmoid()

humour_metric_val_values = []
sarcasm_metric_val_values = []
offensive_metric_val_values = []
motivational_metric_val_values = []
average_metric_val_values = []

for train_idx, test_idx in kfold.split(df_word_cleaned):
  
  print(f'\n\n----------------------------------------TRAINING SPLIT {cur_split}----------------------------------------')
  cur_split+=1


  df_train = df_word_cleaned.iloc[train_idx]
  df_val = df_word_cleaned.iloc[test_idx]
  
  train_dataset = create_dataset(df_train,tokeniser)
  val_dataset = create_dataset(df_val,tokeniser)

  train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE, shuffle=True)
  val_loader = DataLoader(val_dataset,batch_size=BATCH_SIZE)
  
  basic_model = MemotionModel(scratch=False)
  basic_model.load_state_dict(torch.load(MODEL_PATH, map_location= device))
  mtl_model = MtlModel(basic_model).to(device)

  class_weights = ret_class_weights(df_train, train_dataset) 
  class_weights_h = torch.tensor(class_weights[0], dtype=torch.float).to(device)
  class_weights_s = torch.tensor(class_weights[1], dtype=torch.float).to(device)
  class_weights_o = torch.tensor(class_weights[2], dtype=torch.float).to(device)
  class_weights_m = torch.tensor(class_weights[3], dtype=torch.float).to(device)
  
  criterion_h = nn.BCEWithLogitsLoss(pos_weight= class_weights_h)
  criterion_s = nn.BCEWithLogitsLoss(pos_weight= class_weights_s)
  criterion_o = nn.BCEWithLogitsLoss(pos_weight= class_weights_o)
  criterion_m = nn.BCEWithLogitsLoss(pos_weight= class_weights_m)
  optimizer = optim.Adam(mtl_model.parameters(), lr=3e-3,betas=(0.5,0.99))

  for epoch in range(NUM_EPOCHS):  # loop over the dataset multiple times

      # if i%2==0:
      #   break

      # print('Epoch ',epoch+1)


      running_loss = 0.0

      correct_h = 0
      correct_s = 0
      correct_o = 0
      correct_m = 0

      total_h = 0
      total_s = 0
      total_o = 0
      total_m = 0

      losses = []

      loop = tqdm(train_loader)

      for i, x in enumerate(loop):  

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          
          outputs = mtl_model(x['img'].to(device),x['input_ids'].to(device),x['att_mask'].to(device))


          predicted_h = sigmoid_layer(outputs[0].data)
          total_h += x['emotion_humour'].to(device).size(0)

          predicted_s = sigmoid_layer(outputs[1].data)
          total_s += x['emotion_sarcasm'].to(device).size(0)

          predicted_o = sigmoid_layer(outputs[2].data)
          total_o += x['emotion_offensive'].to(device).size(0)


          predicted_m = sigmoid_layer(outputs[3].data)
          # _, predicted_m = torch.max(outputs[3].data, 1)
          total_m += x['emotion_motivational'].to(device).size(0)

          y_true_h = (x['emotion_humour'].to(device))
          y_predicted_h = torch.where(predicted_h>=0.5,1,0).squeeze(-1)
          correct_h += (y_predicted_h.cpu() == x['emotion_humour']).sum().item()

          y_true_s = (x['emotion_sarcasm'].to(device))
          y_predicted_s = torch.where(predicted_s>=0.5,1,0).squeeze(-1)
          correct_s += (y_predicted_s.cpu() == x['emotion_sarcasm']).sum().item()

          y_true_o = (x['emotion_offensive'].to(device))
          y_predicted_o = torch.where(predicted_o>=0.5,1,0).squeeze(-1)
          correct_o += (y_predicted_o.cpu() == x['emotion_offensive']).sum().item()

          y_true_m = (x['emotion_motivational'].to(device))
          y_predicted_m = torch.where(predicted_m>=0.5,1,0).squeeze(-1)
          correct_m += (y_predicted_m.cpu() == x['emotion_motivational']).sum().item()

          loss_h = criterion_h(outputs[0].to(device), x['emotion_humour'].float().unsqueeze(-1).to(device))

          loss_s = criterion_s(outputs[1].to(device), x['emotion_sarcasm'].float().unsqueeze(-1).to(device))

          loss_o = criterion_o(outputs[2].to(device), x['emotion_offensive'].float().unsqueeze(-1).to(device))

          loss_m = criterion_m(outputs[3].to(device), x['emotion_motivational'].float().unsqueeze(-1).to(device))

          loss = loss_h + loss_s + loss_o + loss_m 
          losses.append(loss.item())

          loss.backward()

          optimizer.step()


          # if i%50 == 0:
            
          #   print(f'{epoch+1}:{i+1}')
          #   print(loss)
          #   print(loss_h, loss_m, loss_o, loss_s)
          #   print(y_predicted_h, y_true_h)
          #   print(y_predicted_s, y_true_s)
          #   print(y_predicted_o, y_true_o)
          #   print(y_predicted_m, y_true_m)
          #   print(correct_h/total_h, correct_s/total_s, correct_o/total_o, correct_m/total_m)

          # print(y_true_h.numpy(), y_predicted_h.numpy())
          # print(y_true_s.numpy(), y_predicted_s.numpy())
          # print(y_true_o.numpy(), y_predicted_o.numpy())
          # print(y_true_m.numpy(), y_predicted_m.numpy())

          f1_h = f1_score(y_true_h.cpu().numpy(), y_predicted_h.cpu().numpy(),average='macro')
          f1_s = f1_score(y_true_s.cpu().numpy(), y_predicted_s.cpu().numpy(),average='macro')
          f1_o = f1_score(y_true_o.cpu().numpy(), y_predicted_o.cpu().numpy(),average='macro')
          f1_m = f1_score(y_true_m.cpu().numpy(), y_predicted_m.cpu().numpy(),average='macro')
          f1_avg = (f1_h+ f1_s+ f1_o+ f1_m)/4

          loop.set_description(f"Epoch [{epoch+1}/{NUM_EPOCHS}]")
          loop.set_postfix(loss=sum(losses)/len(losses), f1_humour=f1_h, f1_sarcasm=f1_s, f1_offensive=f1_o, f1_motivational=f1_m, f1_avg =f1_avg  )



  # Validation

  correct_h = 0
  correct_s = 0
  correct_o = 0
  correct_m = 0

  total_h = 0
  total_s = 0
  total_o = 0
  total_m = 0

  y_true_h = []
  y_predicted_h = []

  y_true_s = []
  y_predicted_s = []

  y_true_o = []
  y_predicted_o = []

  y_true_m = []
  y_predicted_m = []

  with torch.no_grad():

    for i, x in enumerate(val_loader):
        
        # if i>2:
        #   break

        outputs = mtl_model(x['img'].float().to(device),x['input_ids'].to(device),x['att_mask'].to(device))
            
        predicted_h = torch.where(nn.Sigmoid()(outputs[0]).data>=0.5,1,0).squeeze(-1)
        total_h += x['emotion_humour'].to(device).size(0)

        predicted_s = torch.where(nn.Sigmoid()(outputs[1]).data>=0.5,1,0).squeeze(-1)
        total_s += x['emotion_sarcasm'].to(device).size(0)

        predicted_o = torch.where(nn.Sigmoid()(outputs[2]).data>=0.5,1,0).squeeze(-1)
        total_o += x['emotion_offensive'].to(device).size(0)

        predicted_m = torch.where(nn.Sigmoid()(outputs[3]).data>=0.5,1,0).squeeze(-1)
        total_m += x['emotion_motivational'].to(device).size(0)

        y_true_h.append(x['emotion_humour'].to(device))
        y_predicted_h.append(predicted_h )
        correct_h += (predicted_h.cpu() == x['emotion_humour']).sum().item()

        y_true_s.append(x['emotion_sarcasm'].to(device))
        y_predicted_s.append(predicted_s)
        correct_s += (predicted_s.cpu() == x['emotion_sarcasm']).sum().item()

        y_true_o.append(x['emotion_offensive'].to(device))
        y_predicted_o.append(predicted_o)
        correct_o += (predicted_o.cpu() == x['emotion_offensive']).sum().item()

        y_true_m.append(x['emotion_motivational'].to(device))
        y_predicted_m.append(predicted_m)
        correct_m += (predicted_m.cpu() == x['emotion_motivational']).sum().item()

        # print(outputs[1])

      # print(y_true_h.numpy(), y_predicted_h.numpy())
      # print(y_true_s.numpy(), y_predicted_s.numpy())
      # print(y_true_o.numpy(), y_predicted_o.numpy())
      # print(y_true_m.numpy(), y_predicted_m.numpy())



    y_true_h = torch.cat(y_true_h,dim=0)
    y_true_s = torch.cat(y_true_s,dim=0)
    y_true_o = torch.cat(y_true_o,dim=0)
    y_true_m = torch.cat(y_true_m,dim=0)


    y_predicted_h = torch.cat(y_predicted_h,dim=0)
    y_predicted_s = torch.cat(y_predicted_s,dim=0)
    y_predicted_o = torch.cat(y_predicted_o,dim=0)
    y_predicted_m = torch.cat(y_predicted_m,dim=0)


    # print(y_true_h.cpu().numpy(), y_predicted_h.cpu().numpy())
    # print(y_true_s.cpu().numpy(), y_predicted_s.cpu().numpy())
    # print(y_true_o.cpu().numpy(), y_predicted_o.cpu().numpy())
    # print(y_true_m.cpu().numpy(), y_predicted_m.cpu().numpy())

    f1_h = f1_score(y_true_h.cpu().numpy(), y_predicted_h.cpu().numpy(),average='macro')
    f1_s = f1_score(y_true_s.cpu().numpy(), y_predicted_s.cpu().numpy(),average='macro')
    f1_o = f1_score(y_true_o.cpu().numpy(), y_predicted_o.cpu().numpy(),average='macro')
    f1_m = f1_score(y_true_m.cpu().numpy(), y_predicted_m.cpu().numpy(),average='macro')
    f1_avg = (f1_h+ f1_s+ f1_o+ f1_m)/4

    humour_metric_val_values.append(f1_h)
    sarcasm_metric_val_values.append(f1_s)
    offensive_metric_val_values.append(f1_o)
    motivational_metric_val_values.append(f1_m)
    average_metric_val_values.append(f1_avg)


    print(f'val_f1_humour={f1_h}, val_f1_sarcasm={f1_s}, val_f1_offensive={f1_o}, val_f1_motivational={f1_m} val_f1_avg={f1_avg}')




----------------------------------------TRAINING SPLIT 1----------------------------------------


Downloading:   0%|          | 0.00/605M [00:00<?, ?B/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

val_f1_humour=0.47181409004667135, val_f1_sarcasm=0.4572002816474594, val_f1_offensive=0.4727920604866369, val_f1_motivational=0.48899423814135223 val_f1_avg=0.47270016758053


In [ ]:
sum(average_metric_val_values)/len(average_metric_val_values)

0.4703050913978288

In [ ]:
torch.save(mtl_model.state_dict(),'/content/drive/MyDrive/Wipro/Final/Task_C/kfold/clip/models/memotion_sentences_taskb.pt')